In [1]:
!pip install langchain langchain_community langchain_groq  faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00


# **MAXIMAL MARGINAL RELEVANCE(MMR)**

### Retrieval Algorithm designed to reduce redudancy in retrieved result while maintaining high relevance with user query

In [2]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

In [3]:
docs = [
    Document(page_content="Python is a popular programming language for AI and data science."),
    Document(page_content="Java is widely used for enterprise software and Android apps."),
    Document(page_content="JavaScript is the language of the web and is used for front-end development."),
    Document(page_content="Python is also used in web development with frameworks like Django and Flask."),
    Document(page_content="C++ is known for high performance and is used in game development."),
    Document(page_content="R is used in statistical computing and graphics, especially in data analysis."),
    Document(page_content="Python supports multiple paradigms including object-oriented and functional programming."),
    Document(page_content="Go is a statically typed language developed at Google, known for concurrency support."),
    Document(page_content="Ruby is appreciated for its elegant syntax and is popular in web development."),
    Document(page_content="Python's extensive libraries make it ideal for machine learning tasks.")
]


In [4]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

<ipython-input-4-98d98bf60d80>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dat

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
vector_store=FAISS.from_documents(docs,embeddings)

In [6]:
normal_retriever=vector_store.as_retriever(search_kwargs={'k':3})

In [7]:
# MMR Retriever
mmr_retriever=vector_store.as_retriever(search_type='mmr',kwargs={'k':2})

In [8]:
query="Tell me about Python and programming languages"
normal_result=normal_retriever.invoke(query)
mmr_result=mmr_retriever.invoke(query)

In [9]:
normal_result

[Document(id='cc279efa-132d-42cc-b801-652d811c0b88', metadata={}, page_content='Python is a popular programming language for AI and data science.'),
 Document(id='6bf4d4ed-9c3d-4028-82db-b2d5bd30a8f2', metadata={}, page_content='Python supports multiple paradigms including object-oriented and functional programming.'),
 Document(id='ab07ae94-99bd-409b-874b-e091f51f8c88', metadata={}, page_content='Python is also used in web development with frameworks like Django and Flask.')]

In [10]:
mmr_result

[Document(id='cc279efa-132d-42cc-b801-652d811c0b88', metadata={}, page_content='Python is a popular programming language for AI and data science.'),
 Document(id='fa381555-d7f3-455c-85dd-de548e7f4de0', metadata={}, page_content='JavaScript is the language of the web and is used for front-end development.'),
 Document(id='6bf4d4ed-9c3d-4028-82db-b2d5bd30a8f2', metadata={}, page_content='Python supports multiple paradigms including object-oriented and functional programming.'),
 Document(id='7d1ef714-3f08-4f3b-8582-041bd62beee5', metadata={}, page_content='Ruby is appreciated for its elegant syntax and is popular in web development.')]

## **Multi-Query Retriever**

### Sometimes our Query is Ambigeous mean to say meaning is not clear.For example - How to stay health?
### In this Method we convert our single query into multiple similar queries using llm and fetch relevan documents for each query in multi-query



In [11]:
documents = [
    Document(page_content="AI powers virtual assistants like Siri, Alexa, and Google Assistant."),
    Document(page_content="Recommendation systems on Netflix and YouTube use AI to suggest content based on your preferences."),
    Document(page_content="Spam filters in email services rely on AI to detect and block unwanted messages."),
    Document(page_content="Self-driving cars use AI for perception, decision-making, and control."),
    Document(page_content="Smart home devices use AI to optimize energy consumption and automate routines."),
    Document(page_content="AI is used in e-commerce for dynamic pricing, product recommendations, and fraud detection."),
    Document(page_content="AI helps doctors with diagnosis by analyzing medical images and patient data."),
    Document(page_content="AI chatbots assist customers on websites by answering common queries."),
    Document(page_content="Face recognition in smartphones and surveillance systems is powered by AI."),
    Document(page_content="Financial institutions use AI to detect unusual transactions and predict credit risks.")
]



In [12]:
from langchain.retrievers.multi_query import MultiQueryRetriever

In [13]:
from langchain_groq import ChatGroq
llm=ChatGroq(model='llama-3.1-8b-instant',api_key='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')

In [14]:
vector_store=FAISS.from_documents(documents,embeddings)

In [15]:
mqr_retriever=MultiQueryRetriever.from_llm(
    retriever=vector_store.as_retriever(search_kwargs={'k':3}),
    llm=llm
)

In [16]:
mqr_result=mqr_retriever.invoke("What are some uses of AI in daily life?")

In [17]:
mqr_result

[Document(id='88725a45-3e07-4706-9cf5-1b5c62266fe2', metadata={}, page_content='AI helps doctors with diagnosis by analyzing medical images and patient data.'),
 Document(id='ff406c9d-554f-4f30-a8f8-4a87a96d8371', metadata={}, page_content='AI chatbots assist customers on websites by answering common queries.'),
 Document(id='6481c105-d259-4074-a093-41e4dcf40012', metadata={}, page_content='Recommendation systems on Netflix and YouTube use AI to suggest content based on your preferences.'),
 Document(id='b6bee649-0f64-4808-b8ab-91bba56f25c0', metadata={}, page_content='AI powers virtual assistants like Siri, Alexa, and Google Assistant.'),
 Document(id='163c5943-e432-4f23-9d32-6adc8c6bc0e3', metadata={}, page_content='Smart home devices use AI to optimize energy consumption and automate routines.')]

## **CONTEXTUAL COMPRESSION RETRIEVER**

### Advance Retriever that improves retrieval quality by compressing documents after retrieval keep only the relevant content(part) from the chunk based on the user query

In [19]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [23]:
vectorstore=FAISS.from_documents(docs,embeddings)

In [24]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [21]:
llm=ChatGroq(model='llama-3.1-8b-instant',api_key='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')

In [25]:
compressor=LLMChainExtractor.from_llm(llm=llm)

In [31]:
compression_retriever=ContextualCompressionRetriever(
                        base_retriever=vectorstore.as_retriever(search_kwargs={'k':3}),
                          base_compressor=compressor
)

In [32]:
query="What is PhotoSynthesis"
result=compression_retriever.invoke(query)

In [33]:
result

[Document(metadata={'source': 'Doc1'}, page_content='Photosynthesis is the process by which green plants convert sunlight into energy.'),
 Document(metadata={'source': 'Doc4'}, page_content='Photosynthesis does not occur in animal cells.'),
 Document(metadata={'source': 'Doc2'}, page_content='The chlorophyll in plant cells captures sunlight during photosynthesis.')]